In [9]:
# библиотека, умеющая управлять HTTP запросами (отправлять и получать ответы)
# library to manage HTTP requests
import requests

# библиотека для работы с JSON
# library to interact with JSON
import json

# библиотеки для обработки геоизображений
# libraries to process rasters
import rasterio, gdal, numpy

# library to work with filesystem
# библиотека для работы с файловой системой
import os

In [15]:
# QGIS Style template for quick styling (created in QGIS and saved as .qml)

qml_template = '''
    <!DOCTYPE qgis PUBLIC 'http://mrcc.com/qgis.dtd' 'SYSTEM'>
    <qgis hasScaleBasedVisibilityFlag="0" maxScale="0" minScale="1e+08" version="3.14.1-Pi" styleCategories="AllStyleCategories">
      <flags>
        <Identifiable>1</Identifiable>
        <Removable>1</Removable>
        <Searchable>1</Searchable>
      </flags>
      <temporal enabled="0" mode="0" fetchMode="0">
        <fixedRange>
          <start></start>
          <end></end>
        </fixedRange>
      </temporal>
      <customproperties>
        <property value="false" key="WMSBackgroundLayer"/>
        <property value="false" key="WMSPublishDataSourceUrl"/>
        <property value="0" key="embeddedWidgets/count"/>
        <property value="Value" key="identify/format"/>
      </customproperties>
      <pipe>
        <rasterrenderer nodataColor="" type="multibandcolor" blueBand="3" redBand="1" greenBand="2" opacity="1" alphaBand="-1">
          <rasterTransparency/>
          <minMaxOrigin>
            <limits>CumulativeCut</limits>
            <extent>WholeRaster</extent>
            <statAccuracy>Estimated</statAccuracy>
            <cumulativeCutLower>0.02</cumulativeCutLower>
            <cumulativeCutUpper>0.98</cumulativeCutUpper>
            <stdDevFactor>2</stdDevFactor>
          </minMaxOrigin>
          <redContrastEnhancement>
            <minValue>REDMIN</minValue>
            <maxValue>REDMAX</maxValue>
            <algorithm>StretchToMinimumMaximum</algorithm>
          </redContrastEnhancement>
          <greenContrastEnhancement>
            <minValue>GREENMIN</minValue>
            <maxValue>GREENMAX</maxValue>
            <algorithm>StretchToMinimumMaximum</algorithm>
          </greenContrastEnhancement>
          <blueContrastEnhancement>
            <minValue>BLUEMIN</minValue>
            <maxValue>BLUEMAX</maxValue>
            <algorithm>StretchToMinimumMaximum</algorithm>
          </blueContrastEnhancement>
        </rasterrenderer>
        <brightnesscontrast contrast="0" brightness="0"/>
        <huesaturation grayscaleMode="0" saturation="0" colorizeStrength="100" colorizeBlue="128" colorizeGreen="128" colorizeOn="0" colorizeRed="255"/>
        <rasterresampler maxOversampling="2"/>
      </pipe>
      <blendMode>0</blendMode>
    </qgis>
    '''

In [11]:
data_directory = 'data'

ngw_host = 'https://test-premium.nextgis.com'
ngw_user = 'webinar'
ngw_password = 'webinar00'
parent_resource = 259

In [20]:
# Создаем группу ресурсов
# Create resource group
request_url = '%s/api/resource/' % ngw_host
# Структура нового объекта
# New object structure 
data = {
    "resource":
    {
        "cls":"resource_group",
        "parent":
        {
            "id": parent_resource # родительский ресурс | parent resource
        },
        "display_name": u'Спутниковый мониторинг', # имя новой группы ресурсов | name for new resource group
        "description":""
    }
}

# отправляем запрос
# send request
r = requests.post(request_url,
                  data=json.dumps(data),
                  auth=(ngw_user, ngw_password))

#new_resource_group_id = r.json()['id']

In [21]:
r.status_code

422

In [22]:
r.content

b'{"title": "Resource display name is not unique", "message": "Resource with same display name already exists (id = 343).", "detail": "Within a single parent resource, each resource must have unique display name. Give the resource a different display name or rename existing.", "exception": "nextgisweb.resource.exception.DisplayNameNotUnique", "status_code": 422, "data": {"serializer": "resource", "resource_id": 343}, "guru_meditation": "dc6bce34b16aa6eb7d7e6b72069eb369"}'

In [13]:
# Получаем список сцен в папке
# get list of satellite scenes in directory
datasets = os.listdir(data_directory)

In [14]:
datasets

['LC08_L1TP_184018_20170331_20200904_02_T1',
 'LC08_L1TP_184018_20180419_20201015_02_T1',
 'LC08_L1TP_184018_20190406_20200829_02_T1',
 'LC08_L1TP_184018_20200408_20200822_02_T1',
 'LC08_L1TP_185018_20210418_20210424_02_T1']

In [16]:
# обходим поочередно все сцены
# walk throug all satellite scenes
scenes_list = []

for dataset in datasets:
    print (u'Обработка сцены (processing scene) %s' % dataset)
    cdate = dataset.split('_')[3]
    scene_name = 'Снимок от (Scene date) %s' % cdate
    # получаем список файлов снимка
    # get list of files of scene
    current_files = os.listdir(os.path.join(data_directory, dataset))
    for cfile in current_files:
        # Получаем красный, синий и зелёный каналы снимка
        # Get red, blue and green channels
        if cfile.find('B2') != -1:
            blue_band_file = os.path.join(data_directory, dataset, cfile)
        if cfile.find('B3') != -1:
            green_band_file = os.path.join(data_directory, dataset, cfile)
        if cfile.find('B4') != -1:
            red_band_file = os.path.join(data_directory, dataset, cfile)
    
    #######################################
    # Создаём RGB-композит из данных снимка
    # Create RGB-composite from scene data
    #######################################
    blue_band=rasterio.open(blue_band_file)
    green_band=rasterio.open(green_band_file)
    red_band=rasterio.open(red_band_file)
    
    band_geo = blue_band.profile
    band_geo.update({"count": 3})
    output_rgb = os.path.join(data_directory, dataset, 'RGB_%s.tif' % cdate)
    output_rgb_clipped = os.path.join(data_directory, dataset, 'RGB_%s_clipped.tif' % cdate)
    
    print ('Создание цветного композита (Create color composite)')
    with rasterio.open(output_rgb, 'w', **band_geo) as dest:
        dest.write(red_band.read(1),1)
        dest.write(green_band.read(1),2)
        dest.write(blue_band.read(1),3)
    
    print ('Перепроецирование (Reprojecting)')
    reprojected = gdal.Warp ('',output_rgb,format='MEM',dstSRS='EPSG:3857')
    print ('Обрезка (Cutting)')
    gdal.Translate(output_rgb_clipped, reprojected, projWin = [3352737.6, 8400399.7, 3387671.7, 8375274.4])
    print ('Расчёт статистики растра (count raster statistics)')
    ds = rasterio.open(output_rgb_clipped).read()
    blue_band_min = numpy.percentile(ds[2],2)
    blue_band_max = numpy.percentile(ds[2],98)
    green_band_min = numpy.percentile(ds[1],2)
    green_band_max = numpy.percentile(ds[1],98)
    red_band_min = numpy.percentile(ds[0],2)
    red_band_max = numpy.percentile(ds[0],98)
    del ds
    
    #######################################
    #######################################
    
    #######################################
    # Загружаем в NGW 
    # Upload to NGW
    #######################################
    # 1. Загрузка файла .TIF в NGW
    # 1. Uploading .TIF to NGW
    print (u'Загрузка растра в NGW (Upload raster to NGW)')
    files = open(output_rgb_clipped, 'rb').read()
    url = "%s/api/component/file_upload/upload" % (ngw_host)
    r = requests.put(url, data=files, auth=(ngw_user, ngw_password))
    response = r.json()
    
    # 2. Создание растрового слоя на базе этого файла
    # 2. Creating raster layer based on this file
    print (u'Создание растрового слоя в NGW (Creating raster layer in NGW)')
    resource = {
        "resource": {
            "cls": "raster_layer",
            "display_name": scene_name,
            "parent": {"id": int(new_resource_group_id)}
        },
        "raster_layer": {
            "source": {
                "id": response['id'],
                "mime_type": response['mime_type'],
                "size": response['size']
            },
            "srs": {"id": 3857}
        }
    }

    url = "%s/api/resource/" % (ngw_host)
    r = requests.post(url, data=json.dumps(resource), auth=(ngw_user, ngw_password))
    new_raster_resource_id = r.json()['id']
    
    # 3. Создание QML-стиля
    # 3. Creating QML-stype
    # Подстановка статистик каналов в содержимое QML
    # Writing raster statistics to QML template
    qml_content = qml_template.replace('REDMIN',str(red_band_min)).replace('REDMAX',str(red_band_max)).replace('BLUEMIN',str(blue_band_min)).replace('BLUEMAX',str(blue_band_max)).replace('GREENMIN',str(green_band_min)).replace('GREENMAX',str(green_band_max))
    
    # 4. Загрузка стиля QML в NGW
    # 4. Uploading QML style to NGW
    print (u'Загрузка стиля в NGW (Upload style to NGW)')
    
    url = "%s/api/component/file_upload/upload" % (ngw_host)
    r = requests.put(url, data=qml_content, auth=(ngw_user, ngw_password))
    response = r.json()
    
    # 5. Создание ресурса стиля на базе этого файла
    # 5. Creating style resource based on this file
    print (u'Создание ресурса стиля в NGW (Creating style resource in NGW)')
    resource = {
        'resource':
            {'cls': 'qgis_raster_style',
            'parent': {
                'id': new_raster_resource_id
            },
            'display_name': scene_name
            },
            'resmeta':
                {'items':
                    {}
                },
            'qgis_raster_style': {
                'file_upload': {
                    "id": response['id'],
                    "mime_type": response['mime_type'],
                    "size": response['size']
                }
            }
        }
    url = "%s/api/resource/" % (ngw_host)
    r = requests.post(url, data=json.dumps(resource), auth=(ngw_user, ngw_password))
    new_style_id = r.json()['id']
    
    # формируем список обработанных сцен, чтобы потом сделать карту
    # create list of processed scenes for future map
    scenes_list.append ({'style_id':new_style_id, 'name': scene_name})
    print ('===========================')

Обработка сцены LC08_L1TP_184018_20170331_20200904_02_T1
Создание цветного композита
Перепроецирование
Обрезка
Расчёт статистики растра
Загрузка растра в NGW
Создание растрового слоя в NGW
Загрузка стиля в NGW
Создание ресурса стиля в NGW
Обработка сцены LC08_L1TP_184018_20180419_20201015_02_T1
Создание цветного композита
Перепроецирование
Обрезка
Расчёт статистики растра
Загрузка растра в NGW
Создание растрового слоя в NGW
Загрузка стиля в NGW
Создание ресурса стиля в NGW
Обработка сцены LC08_L1TP_184018_20190406_20200829_02_T1
Создание цветного композита
Перепроецирование
Обрезка
Расчёт статистики растра
Загрузка растра в NGW
Создание растрового слоя в NGW
Загрузка стиля в NGW
Создание ресурса стиля в NGW
Обработка сцены LC08_L1TP_184018_20200408_20200822_02_T1
Создание цветного композита
Перепроецирование
Обрезка
Расчёт статистики растра
Загрузка растра в NGW
Создание растрового слоя в NGW
Загрузка стиля в NGW
Создание ресурса стиля в NGW
Обработка сцены LC08_L1TP_185018_20210418_20

In [17]:
scenes_list

[{'style_id': 346, 'name': 'Снимок от 20170331'},
 {'style_id': 348, 'name': 'Снимок от 20180419'},
 {'style_id': 350, 'name': 'Снимок от 20190406'},
 {'style_id': 352, 'name': 'Снимок от 20200408'},
 {'style_id': 354, 'name': 'Снимок от 20210418'}]

In [18]:
# Создание карты со всеми загруженными растрами
# Creating map with all uploaded rasters
request_url = '%s/api/resource/' % ngw_host
data = {
        "resource": {
            "display_name": u"Карта спутникового мониторинга (Satellite monitoring map)",
            "parent": {
                "id": new_resource_group_id
            },
            "cls": "webmap"
        },
        "webmap": {
            "extent_left": 30.1,
            "extent_right": 30.4,
            "extent_bottom": 59.8,
            "extent_top": 60.1,
            "root_item": {
                "item_type": "root",
                "children": []
            }
        }
    }

for scene in scenes_list:
    child = {
        "layer_enabled": True,
        "layer_adapter": "image",
        "display_name": scene['name'],
        "layer_style_id": scene['style_id'],
        "item_type": "layer"
    }
    
    data['webmap']['root_item']['children'].append(child)

# отправляем запрос
# send request
r = requests.post(request_url,
                  data=json.dumps(data),
                  auth=(ngw_user, ngw_password))